Carregando os dados necessários

In [43]:
from pandas import DataFrame, read_csv, to_datetime
from os import path

datadir = 'C:/Users/User/Desktop/data_science/mentoria_data_mundo/projeto_ecommerce/_data/clean'

df_clientes = read_csv(path.join(datadir, 'e_commerce_clientes.csv'))
df_carrinhos = read_csv(path.join(datadir, 'e_commerce_carrinhos.csv'))
df_pedidos = read_csv(path.join(datadir, 'e_commerce_pedidos.csv'))

Realizando os tratamentos dos dados necessários

In [52]:
# Do dataframe 'df_pedidos' Iremos pegar a coluna 'total_pago' e a coluna 'id_cliente', assim teremos o valor total pago por cliente, além da data.

# A ideia é dividir em dois grupos, o ticket médio dos clientes em datas normais, e em datas especiais.


# Trocando as ',' por '.' para converter para float na coluna 'total_pago'
df_pedidos['total_pago'] = df_pedidos['total_pago'].replace(',', '.', regex=True).astype({'total_pago': float})

# Trocando o tipo da coluna 'data' para datetime (formato do pandas para datas)
df_pedidos['data'] = to_datetime(df_pedidos['data'], format='%d/%m/%Y %H:%M:%S')

# Filtrando apenas as colunas necessárias do dataframe 'df_pedidos'
df_pedidos.filter(['id_cliente', 'total_pago', 'data']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_cliente  145 non-null    object        
 1   total_pago  145 non-null    float64       
 2   data        145 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 3.5+ KB
